In [2]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, OptimizersConfigDiff
import pandas as pd
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
df = pd.read_parquet('embeddings_results_conclusions_585k_cs1024_ov100_qw3-06B.parquet')

In [3]:
client = QdrantClient(
    url=os.getenv("QDRANT_URL"),
    api_key=os.getenv("QDRANT_API_KEY"),
)

In [4]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='library-test'), CollectionDescription(name='index_1')])

In [68]:
collection = "library-test"

In [75]:
client.create_collection(
    collection_name=collection,
    vectors_config=VectorParams(size=128, distance=Distance.COSINE),
)

True

In [76]:
# disable indexing before inserting initial data
client.update_collection(
    collection_name=collection,
    optimizers_config=OptimizersConfigDiff(indexing_threshold=0),
)

True

In [77]:
# take N publications with all their chunks
N = 50_000
#sample = df.loc[df.sample(N).index]
sample = df.loc[df.index[:N]].reset_index()
embeddings = sample.pop('embedding')
len(sample)

613455

In [78]:
D = 128
vectors = tqdm([v[:D] for v in embeddings.values])
payloads = [r.to_dict() for i, r in sample.iterrows()]

  0%|          | 0/613455 [00:00<?, ?it/s]

In [79]:
client.upload_collection("library-test", vectors=vectors, payload=payloads, parallel=12)

In [80]:
# re-enable indexing before inserting initial data
client.update_collection(
    collection_name=collection,
    optimizers_config=OptimizersConfigDiff(indexing_threshold=20_000),
)

True